In [1]:
import sympy
from sympy import Matrix, S, Symbol, symbols, I, zeros, eye
from sympy import simplify, expand, expand_complex, latex
import numpy as np
from IPython.display import Latex
import scipy
import pandas as pd

# Практическое занятие 17
# Компьютерный практикум по алгебре на Python
## Матричные разложения: Холецкого, LDL, LU, QR. Жорданова форма.

### Задание 1.
Построить разложение Холецкого матриц
$$
M1=\left(
\begin{matrix}
1&-3&0\\
-3&-2&10\\
0&10&7
\end{matrix}
\right)
\quad
M2=\left(
\begin{matrix}
18&1 - 2I& -2\\
1 + 2I&4&-3I\\
-2&3I&5
\end{matrix}
\right)
$$
Проверить положительную определенность эрмитовой матрицы.

In [2]:
A = Matrix([[-1, -3, 0],
            [-3, -2, 10],
            [0, 10, 7]])

B = Matrix([[18, 1 - 2*I, -2],
            [1 + 2*I, 4, -3*I],
            [-2, 3*I, 5]])

LA = A.cholesky(hermitian=False)
LB = B.cholesky()

display(Latex(f'L_A = {latex(LA)},\ L_AL_A^T - A= {latex(simplify(LA * LA.T - A))},\\\\\
                B.is\_positive\_definite\ {B.is_positive_definite},\\\\\
                L_B = {latex(LB)},\\\\\
                simplify(L_B) = {latex(simplify(LB))},\\\\\
                simplify(expand(L_B)) = {latex(simplify(expand(LB)))},\\\\\
                simplify(L_B*L_B^H - B) = {latex(simplify(LB * LB.H - B))}'))

<IPython.core.display.Latex object>

### Задание 2.
Построить  LDL разложение для матриц Задания 1.

In [3]:
A = Matrix([[-1, -3, 0],
            [-3, -2, 10],
            [0, 10, 7]])

B = Matrix([[18, 1 - 2*I, -2],
            [1 + 2*I, 4, -3*I],
            [-2, 3*I, 5]])

LA, DA = A.LDLdecomposition(hermitian=False)
LB, DB = B.LDLdecomposition()
LA, DA, LB, DB = [simplify(item) for item in (LA, DA, LB, DB)]

display(Latex("""A = {}\\\\A.is\_positive\_definite\ {}\\\\\
              L_A = {}, D_A = {},\\\\L_AD_AL_A^T = {},\\\\\
              B = {}\\\\\
              B.is\_positive\_definite\ {}\\\\L_B = {}, D_B = {},\\\\\
              L_BD_BL_B^H = {}\
              """.format(*[latex(item) for item in (A, A.is_positive_definite,
                                                    LA, DA, LA * DA * LA.T,
                                                    B, B.is_positive_definite,
                                                    LB, simplify(DB),
                                                    simplify(LB * DB * LB.H))])))

<IPython.core.display.Latex object>

### Задание  3.
Построить  LU разложение для матрицы
$$
V=\left(
\begin{matrix}
5&-2 - I&3 - 4I&1 + 4I\\
1 - I&-2&5 - I&2 - I\\
5&6 + I&0&5
\end{matrix}
\right)
$$

In [4]:
M = Matrix([[5, -2 - I, 3 - 4 * I, 1 + 4*I],
            [1 - I, -2, 5 - I, 2 - I],
            [5, 6 + I, 0, 5]])

L, U, perm = M.LUdecomposition()
display(Latex("M = {}\\\\perm = {}, L = {}, U = {}\\\\LU = {}".format(*map(latex, (M, perm, *map(simplify, map(expand, (L, U, L * U))))))))

<IPython.core.display.Latex object>

### Задание  4.
Построить  QR разложение для матрицы
$$
A=\left(
\begin{matrix}
3 + i&  2 & -i & 4 - 2i\\
-2 & -3 &  i & -3 + i\\
1 + i & -1 &  0 & 1 - i\\
-1 + i &  -4 & i & -2
\end{matrix}
\right)
$$
показать, что $A = QR$.

In [22]:
M = Matrix([[3+I, 2, -I, 4-2*I],
            [-2, -3, I, -3+I],
            [1+I, -1, 0, 1-I],
            [-1+I, -4, I, -2]])

Q, R = M.QRdecomposition()
Q, R, MQR = [simplify(expand(item)) for item in (Q, R, Q * R)]

Q = Q.col_del(2)
R = R.row_del(2)

QR = Q * R

display(QR)
display(simplify(QR) == M)

Matrix([
[ 3 + I, -1/3 - 4*I/3 + (3 + I)*(5 + I)/6,   (-1 - 4*I)*(1 - 4*I)/51 - I*(3 + I)/3,  (-1 - 4*I)*(11 + 7*I)/51 + (3 - 2*I)*(3 + I)/3],
[    -2,                               -3,           (-4 + I)*(1 - 4*I)/51 + 2*I/3,             -2 + (-4 + I)*(11 + 7*I)/51 + 4*I/3],
[ 1 + I,     -5/3 - I + (1 + I)*(5 + I)/6,  -I*(1 + I)/3 + (-5 - 3*I)*(1 - 4*I)/51,  (-5 - 3*I)*(11 + 7*I)/51 + (1 + I)*(3 - 2*I)/3],
[-1 + I,  -3 - 2*I/3 + (-1 + I)*(5 + I)/6, -I*(-1 + I)/3 + (-9 - 2*I)*(1 - 4*I)/51, (-9 - 2*I)*(11 + 7*I)/51 + (-1 + I)*(3 - 2*I)/3]])

True

### Задание  5.
Найти SVD c sympy и numpy
$$
\left(
\begin{matrix}
1 & 0 & 0 & -2\\
0 & 1 & 0 & 1\\
0 & 0 & 3 & 1
\end{matrix}
\right)
$$

In [30]:
A = np.array([[1, 0, 0, -2],
              [0, 1, 0, 1],
              [0, 0, 3, 1]])

np.linalg.svd(A)

## Задача 6*
Считать из файла "matrix_task_6.xlsx" матрицу $A$, построить для нее разложение lu, ldl и qr. Все полученные матрицы записать в файл "matrix_task_6_ans.xlsx" на отдельные листы

In [32]:
from scipy.linalg import lu, ldl, qr

df = pd.read_excel("matrix_task_6.xlsx", header=None)

A = df.values
P, L, U = lu(A)
L_ldl, D_ldl, perm = ldl(A)
Q, R = qr(A)

with pd.ExcelWriter("matrix_task_6_ans.xlsx") as writer:
    pd.DataFrame(L).to_excel(writer, sheet_name='L', index=False, header=False)
    pd.DataFrame(U).to_excel(writer, sheet_name='U', index=False, header=False)
    pd.DataFrame(L_ldl).to_excel(writer, sheet_name='L_ldl', index=False, header=False)
    pd.DataFrame(D_ldl).to_excel(writer, sheet_name='D_ldl', index=False, header=False)
    pd.DataFrame(Q).to_excel(writer, sheet_name='Q', index=False, header=False)
    pd.DataFrame(R).to_excel(writer, sheet_name='R', index=False, header=False)
